Load the environment variables

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

Marking the GraphState

In [2]:
from typing import TypedDict, Annotated, List, Union, Sequence
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

Custom Tools

Tools are interfaces that an agent can use to interact with the world. They combine a few things:
1. The name of the tool
2. A description of what the tool is
3. JSON schema of what the inputs to the tool are
4. The function to call
5. Whether the result of a tool should be returned directly to the user

In [3]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool
import random

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
    """Returns the input as all lower case."""
    return input.lower()

@tool("upper_case", return_direct=True)
def to_upper_case(input:str) -> str:
    """Returns the input as all upper case."""
    return input.upper()

@tool("random_number", return_direct=True)
def generate_random_number(input:str) -> str:
    """Returns a random number between 0-100."""
    return random.randint(0, 100)

tools = [to_lower_case, to_upper_case, generate_random_number]

#generate_random_numner.run('random')
#to_lower_case.run('SaNdEeP')
#to_upper_case.run('sAnDeEp')


Agent - with binding

In [5]:
from langchain import hub
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
model = ChatOpenAI( temperature=0, streaming=True)

# Bind the functions to model
functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

In [ ]:
'''
inputs = {
    "input": "Give me a random number and then write it in words and then convert them into both lower and upper case.",
    "chat_history": [],
    "intermediate_steps": [],
}

agent_outcome = agent_runnable.invoke(inputs)

agent_outcome
'''

Nodes

In [6]:
# Tools Executor
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

In [10]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define logic that will be used to determine which conditional edge to go down
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise we continue
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Defin the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct a ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    print(f"The agent action is {action}")
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    print(f"The tool result is: {response}")
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name = action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}


Define the graph

In [11]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally, we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that adter `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()


Run it!!

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="You are a very helpful assistant")
user_01 = HumanMessage(content="Give me a random number and then write it in words and then convert it into lower case.")
inputs = {
    "messages": [system_message, user_01]
}

app.invoke(inputs)

The agent action is tool='random_number' tool_input={'input': ''}
The tool result is: 68
The agent action is tool='lower_case' tool_input={'input': 'Sixty Eight'}
The tool result is: sixty eight


In [14]:
user_02 = HumanMessage(content="Please write sAnDeEp in upper case.")
inputs = {
    "messages": [system_message, user_02]
}
app.invoke(inputs)

The agent action is tool='upper_case' tool_input={'input': 'sAnDeEp'}
The tool result is: SANDEEP


{'messages': [SystemMessage(content='You are a very helpful assistant'),
  HumanMessage(content='Please write sAnDeEp in upper case.'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"sAnDeEp"}', 'name': 'upper_case'}}, response_metadata={'finish_reason': 'function_call'}, id='run-7e618b49-072e-4be2-acff-24e8d7528b9b-0'),
  FunctionMessage(content='SANDEEP', name='upper_case'),
  AIMessage(content='The input "sAnDeEp" in upper case is "SANDEEP".', response_metadata={'finish_reason': 'stop'}, id='run-df1de947-788d-4521-bdef-1d9326371807-0')]}

In [16]:
user_03 = HumanMessage(content="What is the temperature in Bangalore today?")
inputs = {
    "messages": [system_message, user_03]
}

app.invoke(inputs)

{'messages': [SystemMessage(content='You are a very helpful assistant'),
  HumanMessage(content='What is the temperature in Bangalore today?'),
  AIMessage(content="I'm sorry, but I am not able to provide real-time information such as the current temperature in Bangalore. You can check the weather on a weather website or use a weather app on your phone to get the most up-to-date information.", response_metadata={'finish_reason': 'stop'}, id='run-1f2a9f1e-47ea-48f3-923a-c9baff894cbf-0')]}